In [2]:
import pandas as pd
import yaml
import json
import requests

Processing Healcare data

In [3]:
medquadDf = pd.read_csv("./data/medquad.csv")
medquadDf.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [4]:
medquadDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16412 entries, 0 to 16411
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    16412 non-null  object
 1   answer      16407 non-null  object
 2   source      16412 non-null  object
 3   focus_area  16398 non-null  object
dtypes: object(4)
memory usage: 513.0+ KB


In [5]:
medquadDf.isna().sum()

question       0
answer         5
source         0
focus_area    14
dtype: int64

Drop and change null data

In [8]:
medquadDf = medquadDf.dropna(subset=["answer"])
medquadDf["focus_area"] = medquadDf["focus_area"].fillna("")

In [9]:
medquadDf.isna().sum()

question      0
answer        0
source        0
focus_area    0
dtype: int64

In [10]:
medquadDf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16407 entries, 0 to 16411
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    16407 non-null  object
 1   answer      16407 non-null  object
 2   source      16407 non-null  object
 3   focus_area  16407 non-null  object
dtypes: object(4)
memory usage: 640.9+ KB


Save to csv

In [11]:
medquadDf.to_csv("./data/medquadDf.csv", index=False)

Create and save to json

In [12]:
chatbotData = []

In [13]:
groupData = medquadDf.groupby("focus_area")

for focus_area, group in groupData:
    data = {
        "focus_area": focus_area,
        "question": group["question"].tolist(),
        "answer": group["answer"].tolist()
    }
    chatbotData.append(data)

medquadJson = {"intents": chatbotData}

In [14]:
with open("./data/medquadDf.json", "w") as json_file:
    json.dump(medquadJson, json_file, indent=4)

Processing conversations data

In [16]:
url1 = "https://raw.githubusercontent.com/gunthercox/chatterbot-corpus/master/chatterbot_corpus/data/english/conversations.yml"
response1 = requests.get(url1)

if response1.status_code == 200:
    dataConver = yaml.safe_load(response1.text)
else:
    print(f"Error: {response1.status_code}")

In [17]:
url2 = "https://raw.githubusercontent.com/gunthercox/chatterbot-corpus/master/chatterbot_corpus/data/english/greetings.yml"
response2 = requests.get(url2)

if response2.status_code == 200:
    dataGreet = yaml.safe_load(response2.text)
else:
    print(f"Error: {response2.status_code}")

Save conversations data to Json

In [27]:
conversationsData = {
    "conversations": dataConver["conversations"] + dataGreet["conversations"]
}

In [28]:
with open("./data/conversations.json", "w") as json_file:
    json.dump(conversationsData, json_file, indent=4)


Save conversations data to csv

In [29]:
conversationsList = []

In [30]:
for conversation in dataConver["conversations"]:
    for i in range(len(conversation) - 1):
        inputUser = conversation[i]
        respon = conversation[i + 1]
        conversationsList.append([inputUser, respon])

In [31]:
for conversation in dataGreet["conversations"]:
    for i in range(len(conversation) - 1):
        inputUser = conversation[i]
        respon = conversation[i + 1]
        conversationsList.append([inputUser, respon])

In [34]:
conversationsDf = pd.DataFrame(conversationsList, columns=["inputUser", "respon"])
conversationsDf.head()

,inputUser,respon
0,"Good morning, how are you?","I am doing well, how about you?"
1,"I am doing well, how about you?",I'm also good.
2,I'm also good.,That's good to hear.
3,That's good to hear.,Yes it is.
4,Hello,Hi


In [35]:
conversationsDf.to_csv("./data/conversations.csv", index=False)